In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [70]:
url_inicial = 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/archive/6'
url_root = 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/archive/6'
r=requests.get(url_inicial)

In [71]:
soup = BeautifulSoup(r.text, 'html.parser')

In [72]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('h2', class_='media-heading')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1700',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1701',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1703',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1704',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1705',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1706',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1728',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1727',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1726',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1724',
 'https://www.revistascca.unam.mx/rica/index.php/rica/issue/view/1723']

In [73]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('h3', class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [74]:
vol2

['https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29924',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29925',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29927',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29928',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29852',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29853',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29855',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29856',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29845',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29847',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29848',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29290',
 'https://www.revistascca.unam.mx/rica/index.php/rica/article/view/29309',
 'https://www.revistascca

In [75]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('h3', class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [76]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://www.revistascca.unam.mx/rica/index.php/rica/issue/archive/6


In [77]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

103

In [78]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [79]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Internacional de Contaminación Ambiental'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Ciencias Ambientales'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link btn btn-primary pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [80]:
list_scraper=list_scraper[0:228]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [81]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,ESQUEMAS IMPLÍCITOS BALANCEADOS Y ABSOLUTAMENT...,Se sugieren esquemas en diferencias finitas pa...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
1,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,CONCENTRACION DE METALES PESADOS EN OSTIONES (...,Se determinaron las concentraciones de vanadio...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
2,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,EFECTOS CITOLÓGICOS DE ALGUNOS INSECTICIDAS CA...,Los efectos citológicos de dos insecticidas ca...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
3,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,PROBLEMAS EN LA REGULACIÓN DE SUSTANCIAS CARCI...,Se discute la influencia que tienen factores n...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
4,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,"EFECTOS COMBINADOS DE CADMIO, PLOMO Y COBRE SO...",Los síntomas de toxicidad en los vegetales rel...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
...,...,...,...,...,...,...,...,...
98,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,SISTER CHROMATID EXCHANGES INDUCED BY THINNER ...,Se emplearon células meristemáticas de la raíz...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
99,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,GENOTOXICIDAD DE DIFERENTES CANCERIGENOS A CUL...,Se utilizaron cultivos primarios de hepatocito...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
100,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,"GENETIC EFFECTS OF THINNER, BENZENE AND TOLUEN...",Se estudiaron algunos efectos genéticos de tin...,None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...
101,Revista Internacional de Contaminación Ambiental,I. Físico Matemáticas y Ciencias de la Tierra,Ciencias Ambientales,"GENETIC EFFECTS OF THINNER, BENZENE AND TOLUEN...","Se investigaron los efectos del tíner, del ben...",None,https://www.revistascca.unam.mx/rica/index.php...,https://www.revistascca.unam.mx/rica/index.php...


In [82]:
df_catalogo.to_csv('Revista059.06.csv', index=False)